## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:

In [1]:
NAME = "Δεϊρμεντζόγλου Ιωάννης"
AEM = "10015"

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)

In [2]:
# Step 1: Import the necessary libraries
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Step 2: Load the breast cancer dataset
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# Step 3: Split the dataset into training and test sets with a 70%/30% split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [3]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116

**2** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)

In [4]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        self.base_estimator = estimator
        self.models = []
        self.ignore_indices = []

    def fit(self, X_train, y_train):
        self.models = []
        self.ignore_indices = []
        n_features = X_train.shape[1]

        for i in range(n_features):
            # Clone the base estimator
            model = clone(self.base_estimator)

            # Indices of features to use (ignoring the i-th feature)
            feature_indices = [j for j in range(n_features) if j != i]
            self.ignore_indices.append(feature_indices)

            # Train the model on the reduced feature set
            model.fit(X_train[:, feature_indices], y_train)
            self.models.append(model)

    def predict(self, X):
        # Collect predictions from all models
        predictions = np.zeros((X.shape[0], len(self.models)))

        for i, model in enumerate(self.models):
            feature_indices = self.ignore_indices[i]
            predictions[:, i] = model.predict(X[:, feature_indices])

        # Aggregate the predictions (e.g., by majority voting)
        y_pred = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predictions)

        return y_pred

In [5]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006

**3** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        self.n_estimators = n_estimators
        self.estimator = estimator
        self.estimators = []
        self.estimator_weights = []

    def fit(self, X_train, y_train):
        m = len(X_train)
        weights = np.ones(m) / m  # Initialize example weights

        for t in range(self.n_estimators):
            # Train model on the weighted training set
            estimator = clone(self.estimator)
            estimator.fit(X_train, y_train, sample_weight=weights)

            y_pred = estimator.predict(X_train)
            err = np.sum(weights * (y_pred != y_train)) / np.sum(weights)  # Weighted error

            if err >= 0.5 or err == 0:
                break

            alpha = np.log((1 - err) / err)  # Weight of the current estimator
            self.estimators.append(estimator)
            self.estimator_weights.append(alpha)

            # Update example weights
            weights *= np.exp(alpha * (y_pred != y_train))
            weights /= np.sum(weights)  # Normalization

    def predict(self, X):
        votes = np.zeros((len(X), len(np.unique(y_train))))  # Initialize votes

        for estimator, alpha in zip(self.estimators, self.estimator_weights):
            pred = estimator.predict(X)
            for i, p in enumerate(pred):
                votes[i, p] += alpha  # Weighted vote

        return np.argmax(votes, axis=1)

In [9]:
import warnings
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Suppress the FutureWarning about 'algorithm' deprecation
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

    # Ίδιο αποτέλεσμα και με τη κλάση της sklearn
    ab = AdaBoostClassifier(
        n_estimators=20,
        algorithm="SAMME",  # Deprecated but used for reproducibility
        estimator=DecisionTreeClassifier(max_depth=1, random_state=1)
    )
    ab.fit(X_train, y_train)

assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591


In [10]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591

In [11]:
# Ίδιο αποτέλεσμα και με τη κλάση της sklearn
ab = AdaBoostClassifier(n_estimators=20, algorithm="SAMME", estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
